In [1]:
import cv2
from darkflow.net.build import TFNet
import matplotlib.pyplot as plt

from shutil import copy2
import glob 
import random 
import os
import json
import numpy as np
%config InlineBackend.figure_format = 'svg'

C:\anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
# define the model options and run

options03 = {
    'model': 'cfg/yolo.cfg',
    'load': 'bin/yolov2.weights',
    'threshold': 0.3, # 0.05 for ir
    'gpu': 0
}

options005 = {
    'model': 'cfg/yolo.cfg',
    'load': 'bin/yolov2.weights',
    'threshold': 0.05, # 0.05 for ir
    'gpu': 0
}

tfnet = TFNet(options03)
tfnet_005 = TFNet(options005)

Parsing ./cfg/yolov2.cfg
Parsing cfg/yolo.cfg
Loading bin/yolov2.weights ...
Successfully identified 203934260 bytes
Finished in 0.027950525283813477s
Model has a coco model name, loading coco labels.

Building net ...
Source | Train? | Layer description                | Output size
-------+--------+----------------------------------+---------------
       |        | input                            | (?, 608, 608, 3)
 Load  |  Yep!  | conv 3x3p1_1  +bnorm  leaky      | (?, 608, 608, 32)
 Load  |  Yep!  | maxp 2x2p0_2                     | (?, 304, 304, 32)
 Load  |  Yep!  | conv 3x3p1_1  +bnorm  leaky      | (?, 304, 304, 64)
 Load  |  Yep!  | maxp 2x2p0_2                     | (?, 152, 152, 64)
 Load  |  Yep!  | conv 3x3p1_1  +bnorm  leaky      | (?, 152, 152, 128)
 Load  |  Yep!  | conv 1x1p0_1  +bnorm  leaky      | (?, 152, 152, 64)
 Load  |  Yep!  | conv 3x3p1_1  +bnorm  leaky      | (?, 152, 152, 128)
 Load  |  Yep!  | maxp 2x2p0_2                     | (?, 76, 76, 128)
 Load  | 

In [3]:
 pass

In [4]:
random.seed(0)

def getBBox(inp_dir,out_dir):
#     print(inp_dir)
#     print(os.path.dirname(out_dir))
    img = cv2.imread(inp_dir, cv2.IMREAD_COLOR)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    # use YOLO to predict the image
     
    result = tfnet_005.return_predict(img)
    for i in result:
        i["confidence"] = float(i["confidence"])
    #result['confidence'] = float(result['confidence'])
#     result.append(os.path.basename(inp_dir))
#     print(type(result))
    # pull out some info from the results

    for i in range(len(result)):
        tl = (result[i]['topleft']['x'], result[i]['topleft']['y'])
        br = (result[i]['bottomright']['x'], result[i]['bottomright']['y'])
        label = result[i]['label']
        
        
        # add the box and label and display it
        img = cv2.rectangle(img, tl, br, (0, 255, 0), 7)
        img = cv2.putText(img, label, tl, cv2.FONT_HERSHEY_COMPLEX, 1, (0, 0, 0), 2)
        
#     if(len(result)==0):
#         result=[{}]
    result.insert(0,{'FileName':os.path.basename(inp_dir)})
    print(result)
    
    cv2.imwrite(out_dir,img)
    print("Image written to",os.path.dirname(out_dir))
     
    
    return result
    
    
# plt.imshow(img)
# plt.show()


In [5]:
def getRandomNumPics(dir,num=250):  
    # print(glob.glob("C:/Users/mhbt-/Desktop/kaistImageSet/DaySet/visible/*.jpg"))
    fileList = glob.glob(dir)
    #fileList = [os.path.basename(x) for x in glob.glob(dir)]
    randlist = random.sample(range(1, len(fileList)), num)
    
    res = [fileList[i] for i in randlist]
    #print(res)
    return res
    # copy2("../sample_img/sample_computer.jpg", "./visible")

def getFileName(dir):
    for i in list(range(len(dir))[::-1]):         
        if(dir[i] == '.'):break
    print(dir[:i])        
    return dir[:i]

In [6]:
SET = "/set05-night"
MAIN_DIR = "C:/Users/mhbt-/Desktop/kaistImageSet"+SET
OUT_DIR = "C:/Users/mhbt-/Desktop/gitStuffs/darkflow/kaistimages/output"+SET
#typeFolders = ['lwir','visible']
typeFolders = ['lwir']

ls = glob.glob(MAIN_DIR+"/*")
out_dir = [os.path.basename(x) for x in glob.glob(OUT_DIR+"/*")] 


for dir in ls:     
    tempdir=dir+"/"+typeFolders[0]+"/*.jpg"
    fileNameList = [os.path.basename(x)+'\n' for x in getRandomNumPics(tempdir)]
    pics = getRandomNumPics(tempdir,num=250)
    
    #print(fileNameList)
    
    for type in typeFolders:
        out_dir = OUT_DIR+"/"+os.path.basename(dir)+"/"+type+"_005"
        if not os.path.exists(out_dir):
            os.makedirs(out_dir) 
        file = open(out_dir+"/picName.txt","w") 
        file.write(''.join(fileNameList)) 
        file.close()
        
        resList=[]
    
        for fname in fileNameList:
            print("LENGTH---------------", len(fileNameList))
            fname = fname.strip('\n')
            result = getBBox(MAIN_DIR+"/"+os.path.basename(dir)+"/"+type+"/"+fname,out_dir+"/"+fname)
            resList.append(result)
        with open(os.path.dirname(out_dir+"/"+fname)+"/result.json", 'w') as fout:
            for res in resList:
                json.dump(res, fout,indent=4)
            
    





LENGTH--------------- 250
[{'FileName': 'I01578.jpg'}, {'label': 'person', 'confidence': 0.11644620448350906, 'topleft': {'x': 136, 'y': 135}, 'bottomright': {'x': 191, 'y': 208}}, {'label': 'person', 'confidence': 0.12194731831550598, 'topleft': {'x': 243, 'y': 203}, 'bottomright': {'x': 278, 'y': 260}}, {'label': 'person', 'confidence': 0.4271451234817505, 'topleft': {'x': 154, 'y': 226}, 'bottomright': {'x': 190, 'y': 286}}, {'label': 'person', 'confidence': 0.2060771882534027, 'topleft': {'x': 195, 'y': 230}, 'bottomright': {'x': 232, 'y': 281}}, {'label': 'person', 'confidence': 0.2587999999523163, 'topleft': {'x': 260, 'y': 212}, 'bottomright': {'x': 294, 'y': 300}}, {'label': 'person', 'confidence': 0.28942450881004333, 'topleft': {'x': 240, 'y': 227}, 'bottomright': {'x': 270, 'y': 337}}, {'label': 'person', 'confidence': 0.6048424243927002, 'topleft': {'x': 292, 'y': 223}, 'bottomright': {'x': 333, 'y': 359}}, {'label': 'person', 'confidence': 0.06149827316403389, 'topleft': {

[{'FileName': 'I02390.jpg'}]
Image written to C:/Users/mhbt-/Desktop/gitStuffs/darkflow/kaistimages/output/set05-night/V000/lwir_005
LENGTH--------------- 250
[{'FileName': 'I00895.jpg'}]
Image written to C:/Users/mhbt-/Desktop/gitStuffs/darkflow/kaistimages/output/set05-night/V000/lwir_005
LENGTH--------------- 250
[{'FileName': 'I02068.jpg'}]
Image written to C:/Users/mhbt-/Desktop/gitStuffs/darkflow/kaistimages/output/set05-night/V000/lwir_005
LENGTH--------------- 250
[{'FileName': 'I00571.jpg'}, {'label': 'person', 'confidence': 0.7640910148620605, 'topleft': {'x': 106, 'y': 223}, 'bottomright': {'x': 170, 'y': 383}}, {'label': 'person', 'confidence': 0.8163132667541504, 'topleft': {'x': 183, 'y': 228}, 'bottomright': {'x': 244, 'y': 386}}, {'label': 'handbag', 'confidence': 0.28784576058387756, 'topleft': {'x': 161, 'y': 314}, 'bottomright': {'x': 203, 'y': 374}}, {'label': 'suitcase', 'confidence': 0.30174458026885986, 'topleft': {'x': 167, 'y': 318}, 'bottomright': {'x': 205, '

[{'FileName': 'I00303.jpg'}, {'label': 'person', 'confidence': 0.15871268510818481, 'topleft': {'x': 288, 'y': 222}, 'bottomright': {'x': 356, 'y': 282}}, {'label': 'person', 'confidence': 0.14082936942577362, 'topleft': {'x': 412, 'y': 217}, 'bottomright': {'x': 445, 'y': 283}}, {'label': 'person', 'confidence': 0.2935124933719635, 'topleft': {'x': 433, 'y': 215}, 'bottomright': {'x': 465, 'y': 284}}, {'label': 'person', 'confidence': 0.28821220993995667, 'topleft': {'x': 471, 'y': 217}, 'bottomright': {'x': 505, 'y': 286}}, {'label': 'person', 'confidence': 0.1179528534412384, 'topleft': {'x': 503, 'y': 216}, 'bottomright': {'x': 538, 'y': 287}}, {'label': 'person', 'confidence': 0.06077772006392479, 'topleft': {'x': 524, 'y': 222}, 'bottomright': {'x': 571, 'y': 286}}]
Image written to C:/Users/mhbt-/Desktop/gitStuffs/darkflow/kaistimages/output/set05-night/V000/lwir_005
LENGTH--------------- 250
[{'FileName': 'I02802.jpg'}, {'label': 'car', 'confidence': 0.08075150847434998, 'tople

[{'FileName': 'I02624.jpg'}, {'label': 'car', 'confidence': 0.3218349814414978, 'topleft': {'x': 252, 'y': 208}, 'bottomright': {'x': 320, 'y': 242}}, {'label': 'car', 'confidence': 0.4665098488330841, 'topleft': {'x': 101, 'y': 240}, 'bottomright': {'x': 160, 'y': 277}}, {'label': 'car', 'confidence': 0.12699179351329803, 'topleft': {'x': 170, 'y': 238}, 'bottomright': {'x': 201, 'y': 264}}, {'label': 'car', 'confidence': 0.35043248534202576, 'topleft': {'x': 152, 'y': 235}, 'bottomright': {'x': 209, 'y': 271}}, {'label': 'car', 'confidence': 0.058152832090854645, 'topleft': {'x': 186, 'y': 230}, 'bottomright': {'x': 243, 'y': 268}}]
Image written to C:/Users/mhbt-/Desktop/gitStuffs/darkflow/kaistimages/output/set05-night/V000/lwir_005
LENGTH--------------- 250
[{'FileName': 'I00838.jpg'}, {'label': 'person', 'confidence': 0.06600232422351837, 'topleft': {'x': 7, 'y': 293}, 'bottomright': {'x': 143, 'y': 491}}]
Image written to C:/Users/mhbt-/Desktop/gitStuffs/darkflow/kaistimages/out

[{'FileName': 'I02562.jpg'}, {'label': 'person', 'confidence': 0.09532612562179565, 'topleft': {'x': 92, 'y': 195}, 'bottomright': {'x': 145, 'y': 269}}, {'label': 'person', 'confidence': 0.05143968015909195, 'topleft': {'x': 349, 'y': 211}, 'bottomright': {'x': 365, 'y': 233}}, {'label': 'person', 'confidence': 0.07734579592943192, 'topleft': {'x': 417, 'y': 231}, 'bottomright': {'x': 449, 'y': 278}}, {'label': 'person', 'confidence': 0.14045225083827972, 'topleft': {'x': 436, 'y': 223}, 'bottomright': {'x': 474, 'y': 294}}, {'label': 'person', 'confidence': 0.06297121942043304, 'topleft': {'x': 461, 'y': 229}, 'bottomright': {'x': 508, 'y': 295}}, {'label': 'person', 'confidence': 0.12922237813472748, 'topleft': {'x': 509, 'y': 233}, 'bottomright': {'x': 607, 'y': 344}}, {'label': 'car', 'confidence': 0.06524024158716202, 'topleft': {'x': 229, 'y': 195}, 'bottomright': {'x': 295, 'y': 226}}, {'label': 'car', 'confidence': 0.116925448179245, 'topleft': {'x': 258, 'y': 205}, 'bottomrig

...

[{'FileName': 'I00834.jpg'}, {'label': 'person', 'confidence': 0.09264766424894333, 'topleft': {'x': 8, 'y': 294}, 'bottomright': {'x': 151, 'y': 490}}, {'label': 'boat', 'confidence': 0.07999352365732193, 'topleft': {'x': 285, 'y': 211}, 'bottomright': {'x': 405, 'y': 314}}, {'label': 'cup', 'confidence': 0.05032866820693016, 'topleft': {'x': 287, 'y': 218}, 'bottomright': {'x': 406, 'y': 340}}]
Image written to C:/Users/mhbt-/Desktop/gitStuffs/darkflow/kaistimages/output/set05-night/V000/lwir_005
LENGTH--------------- 250
[{'FileName': 'I00238.jpg'}]
Image written to C:/Users/mhbt-/Desktop/gitStuffs/darkflow/kaistimages/output/set05-night/V000/lwir_005
LENGTH--------------- 250
[{'FileName': 'I02770.jpg'}, {'label': 'boat', 'confidence': 0.4035165309906006, 'topleft': {'x': 243, 'y': 204}, 'bottomright': {'x': 327, 'y': 260}}, {'label': 'boat', 'confidence': 0.5294809937477112, 'topleft': {'x': 190, 'y': 209}, 'bottomright': {'x': 259, 'y': 259}}]
Image written to C:/Users/mhbt-/Desk

[{'FileName': 'I02332.jpg'}, {'label': 'person', 'confidence': 0.1142900139093399, 'topleft': {'x': 355, 'y': 233}, 'bottomright': {'x': 418, 'y': 337}}, {'label': 'dog', 'confidence': 0.17757007479667664, 'topleft': {'x': 339, 'y': 226}, 'bottomright': {'x': 430, 'y': 330}}]
Image written to C:/Users/mhbt-/Desktop/gitStuffs/darkflow/kaistimages/output/set05-night/V000/lwir_005
LENGTH--------------- 250
[{'FileName': 'I02084.jpg'}, {'label': 'car', 'confidence': 0.05344158783555031, 'topleft': {'x': 258, 'y': 235}, 'bottomright': {'x': 314, 'y': 279}}]
Image written to C:/Users/mhbt-/Desktop/gitStuffs/darkflow/kaistimages/output/set05-night/V000/lwir_005
LENGTH--------------- 250
[{'FileName': 'I01276.jpg'}]
Image written to C:/Users/mhbt-/Desktop/gitStuffs/darkflow/kaistimages/output/set05-night/V000/lwir_005
LENGTH--------------- 250
[{'FileName': 'I02659.jpg'}, {'label': 'car', 'confidence': 0.11327794939279556, 'topleft': {'x': 161, 'y': 203}, 'bottomright': {'x': 219, 'y': 256}}, 

[{'FileName': 'I00051.jpg'}]
Image written to C:/Users/mhbt-/Desktop/gitStuffs/darkflow/kaistimages/output/set05-night/V000/lwir_005
LENGTH--------------- 250
[{'FileName': 'I01876.jpg'}, {'label': 'person', 'confidence': 0.8283777832984924, 'topleft': {'x': 195, 'y': 251}, 'bottomright': {'x': 266, 'y': 421}}]
Image written to C:/Users/mhbt-/Desktop/gitStuffs/darkflow/kaistimages/output/set05-night/V000/lwir_005
LENGTH--------------- 250
[{'FileName': 'I00324.jpg'}, {'label': 'person', 'confidence': 0.05971762165427208, 'topleft': {'x': 331, 'y': 0}, 'bottomright': {'x': 454, 'y': 188}}, {'label': 'person', 'confidence': 0.10779385268688202, 'topleft': {'x': 185, 'y': 228}, 'bottomright': {'x': 258, 'y': 288}}, {'label': 'person', 'confidence': 0.4565286934375763, 'topleft': {'x': 344, 'y': 217}, 'bottomright': {'x': 374, 'y': 287}}, {'label': 'person', 'confidence': 0.2655014097690582, 'topleft': {'x': 374, 'y': 211}, 'bottomright': {'x': 402, 'y': 288}}, {'label': 'person', 'confide

[{'FileName': 'I02759.jpg'}, {'label': 'person', 'confidence': 0.09273464232683182, 'topleft': {'x': 415, 'y': 209}, 'bottomright': {'x': 431, 'y': 234}}, {'label': 'person', 'confidence': 0.07095108926296234, 'topleft': {'x': 488, 'y': 215}, 'bottomright': {'x': 504, 'y': 256}}, {'label': 'car', 'confidence': 0.050412531942129135, 'topleft': {'x': 234, 'y': 185}, 'bottomright': {'x': 322, 'y': 221}}, {'label': 'car', 'confidence': 0.2540813684463501, 'topleft': {'x': 236, 'y': 204}, 'bottomright': {'x': 325, 'y': 259}}, {'label': 'car', 'confidence': 0.20683087408542633, 'topleft': {'x': 35, 'y': 225}, 'bottomright': {'x': 118, 'y': 275}}, {'label': 'boat', 'confidence': 0.06332685053348541, 'topleft': {'x': 145, 'y': 203}, 'bottomright': {'x': 224, 'y': 255}}, {'label': 'boat', 'confidence': 0.2184847593307495, 'topleft': {'x': 197, 'y': 211}, 'bottomright': {'x': 259, 'y': 256}}]
Image written to C:/Users/mhbt-/Desktop/gitStuffs/darkflow/kaistimages/output/set05-night/V000/lwir_005


[{'FileName': 'I02666.jpg'}, {'label': 'car', 'confidence': 0.06103304773569107, 'topleft': {'x': 165, 'y': 208}, 'bottomright': {'x': 216, 'y': 258}}, {'label': 'car', 'confidence': 0.1575073003768921, 'topleft': {'x': 243, 'y': 208}, 'bottomright': {'x': 318, 'y': 245}}, {'label': 'car', 'confidence': 0.3559945225715637, 'topleft': {'x': 83, 'y': 229}, 'bottomright': {'x': 164, 'y': 285}}, {'label': 'car', 'confidence': 0.15261968970298767, 'topleft': {'x': 2, 'y': 255}, 'bottomright': {'x': 59, 'y': 303}}]
Image written to C:/Users/mhbt-/Desktop/gitStuffs/darkflow/kaistimages/output/set05-night/V000/lwir_005
LENGTH--------------- 250
[{'FileName': 'I00331.jpg'}, {'label': 'person', 'confidence': 0.05492443963885307, 'topleft': {'x': 341, 'y': 216}, 'bottomright': {'x': 360, 'y': 248}}, {'label': 'person', 'confidence': 0.05279017239809036, 'topleft': {'x': 182, 'y': 225}, 'bottomright': {'x': 245, 'y': 287}}, {'label': 'person', 'confidence': 0.26490750908851624, 'topleft': {'x': 31